In [0]:
import numpy as np
import pandas as pd
import random
from numpy.random import seed
seed(125)
from tensorflow import set_random_seed
set_random_seed(125)
import keras
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization

In [0]:
# The most robust way to report results and compare models is to repeat your experiment many times (30+) 
# and use summary statistics.


from numpy.random import seed
seed(125)
from tensorflow import set_random_seed
set_random_seed(125)

In [0]:
# FASHION_MNIST class

class FASHION_MNIST():
  
    def __init__(self,
                 img_rows=28, 
                 img_cols=28,
                 num_classes=10,
                 l1_out=32, 
                 l2_out=64,
                 l3_out=128,
                 l4_out=128,
                 l1_drop=0.25, 
                 l2_drop=0.25,
                 l3_drop=0.4,
                 l4_drop=0.3,
                 batch_size=256, 
                 epochs=50, 
                 validation_split=0.2):
      
        self.__img_rows = img_rows
        self.__img_cols = img_cols
        self.__num_classes = num_classes
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l3_out = l3_out
        self.l4_out = l4_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.l3_drop = l3_drop
        self.l4_drop = l4_drop        
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__X_train, self.__X_test, self.__y_train, self.__y_test = self.load_process_data()
        self.__model = self.fashion_mnist_model()
        

    # load and process Fashion MNIST data (can load from Keras)  
    def load_process_data(self):
      
        (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

        # Preprocess input data for Keras

        # Each image is 28x28 pixels
        # Specify input shape sutable for Keras
        # to be used as input image dimensions
        input_shape = (self.__img_rows, self.__img_cols, 1)

        # When using the Theano backend, you must explicitly declare a dimension for the depth of the input image. 
        # For example, a full-color image with all 3 RGB channels will have a depth of 3
        # As the fashion mnist images are in black and white, only have a depth of 1

        # Reshape input data
        # Want to transform the dataset from having shape (n, width, height) to (n, width, height, depth)
        X_train = X_train.reshape(X_train.shape[0], self.__img_rows, self.__img_rows, 1)
        X_test = X_test.reshape(X_test.shape[0], self.__img_rows, self.__img_cols, 1)

        # Convert input data to float32
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')

        # Normalize data values to the range [0,1]
        # note: Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black)
        X_train /= 255
        X_test /= 255

        # For the output, we change number to one-hot vector. 
        y_train = to_categorical(y_train, 10)
        y_test = to_categorical(y_test, 10)

        return X_train, X_test, y_train, y_test

      
    # specify Fashion MNIST model architecture
    def fashion_mnist_model(self):
      
        model = Sequential()        
        
        # First convolution layer
        model.add(Conv2D(filters=self.l1_out, 
                         kernel_size=(3, 3),
                         kernel_initializer='he_normal',
                         input_shape=(self.__img_rows, self.__img_cols, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(self.l1_drop))

        # Second convolution layer
        model.add(Conv2D(filters=self.l2_out, 
                         kernel_size=(3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(self.l2_drop))
        
        # Third convolution layer
        model.add(Conv2D(filters=self.l3_out, 
                         kernel_size=(3, 3)))
        model.add(Activation('relu'))
        model.add(Dropout(self.l3_drop))

        # Dense layer
        model.add(Flatten())
        model.add(Dense(units=self.l4_out))
        model.add(Activation('relu'))
        model.add(Dropout(self.l4_drop))
        
        model.add(Dense(self.__num_classes))
        model.add(Activation('softmax'))
        
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])
        
        return model

    
    # fit mnist model
    def fit(self):
      
        early_stopping = EarlyStopping(patience=4, verbose=1)
        
        self.__model.fit(self.__X_train, self.__y_train,
                                     batch_size=self.batch_size,
                                     epochs=self.epochs,
                                     verbose=1,
                                     validation_split=self.validation_split,
                                     callbacks=[early_stopping])
    
    # evaluate mnist model
    def evaluate(self):
      
        self.fit()
        
        evaluation = self.__model.evaluate(self.__X_test,
                                           self.__y_test,
                                           batch_size=self.batch_size,
                                           verbose=0)
        
        return evaluation
      
      

### Run Basic model

In [0]:
# Function to run FASHION_MNIST class

def run_fashion_mnist(img_rows=28, 
                      img_cols=28,
                      num_classes=10,
                      l1_out=32,
                      l2_out=64,
                      l3_out=128,
                      l4_out=128,
                      l1_drop=0.25,
                      l2_drop=0.25,
                      l3_drop=0.4,
                      l4_drop=0.3,
                      batch_size=256, 
                      epochs=50, 
                      validation_split=0.2):
    
    fashion_mnist = FASHION_MNIST(img_rows= img_rows,
                                  img_cols = img_cols,
                                  num_classes = num_classes,
                                  l1_out=l1_out,
                                  l2_out=l2_out,
                                  l3_out=l3_out,
                                  l4_out=l4_out,
                                  l1_drop=l1_drop,
                                  l2_drop=l2_drop,
                                  l3_drop=l3_drop,
                                  l4_drop=l4_drop,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  validation_split=validation_split)
    
    evaluation = fashion_mnist.evaluate()
    
    return evaluation

In [6]:

import time

start = time.time()

evaluation = run_fashion_mnist()

end = time.time()
print("time elapsed: {}".format(end - start))



4423680/4422102 [==============================] - 3s 1us/step
Train on 48000 samples, validate on 12000 samples
Epoch 1/50
46336/48000 [===========================>..] - ETA: 0s - loss: 0.8375 - acc: 0.6884

48000/48000 [==============================] - 6s 133us/step - loss: 0.8282 - acc: 0.6916 - val_loss: 0.5057 - val_acc: 0.8148
Epoch 2/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.5073 - acc: 0.8105 - val_loss: 0.4049 - val_acc: 0.8523
Epoch 3/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.4326 - acc: 0.8395 - val_loss: 0.3563 - val_acc: 0.8646
Epoch 4/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.3891 - acc: 0.8578 - val_loss: 0.3217 - val_acc: 0.8799
Epoch 5/50
36352/48000 [=====================>........] - ETA: 0s - loss: 0.3617 - acc: 0.8667

48000/48000 [==============================] - 4s 85us/step - loss: 0.3618 - acc: 0.8670 - val_loss: 0.3066 - val_acc: 0.8843
Epoch 6/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.3384 - acc: 0.8747 - val_loss: 0.2868 - val_acc: 0.8910
Epoch 7/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.3242 - acc: 0.8814 - val_loss: 0.2821 - val_acc: 0.8938
Epoch 8/50
48000/48000 [==============================] - 4s 87us/step - loss: 0.3093 - acc: 0.8882 - val_loss: 0.2742 - val_acc: 0.8988
Epoch 9/50
35584/48000 [=====================>........] - ETA: 1s - loss: 0.2995 - acc: 0.8913

48000/48000 [==============================] - 4s 86us/step - loss: 0.2991 - acc: 0.8911 - val_loss: 0.2687 - val_acc: 0.9003
Epoch 10/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.2912 - acc: 0.8920 - val_loss: 0.2532 - val_acc: 0.9062
Epoch 11/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.2857 - acc: 0.8950 - val_loss: 0.2516 - val_acc: 0.9079
Epoch 12/50
48000/48000 [==============================] - 4s 87us/step - loss: 0.2750 - acc: 0.8984 - val_loss: 0.2455 - val_acc: 0.9086
Epoch 13/50
34816/48000 [====================>.........] - ETA: 1s - loss: 0.2692 - acc: 0.9010

48000/48000 [==============================] - 4s 86us/step - loss: 0.2694 - acc: 0.9007 - val_loss: 0.2457 - val_acc: 0.9105
Epoch 14/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.2638 - acc: 0.9040 - val_loss: 0.2397 - val_acc: 0.9103
Epoch 15/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2599 - acc: 0.9035 - val_loss: 0.2413 - val_acc: 0.9106
Epoch 16/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2541 - acc: 0.9066 - val_loss: 0.2315 - val_acc: 0.9148
Epoch 17/50
34816/48000 [====================>.........] - ETA: 1s - loss: 0.2506 - acc: 0.9062

48000/48000 [==============================] - 4s 86us/step - loss: 0.2474 - acc: 0.9079 - val_loss: 0.2346 - val_acc: 0.9121
Epoch 18/50
48000/48000 [==============================] - 4s 87us/step - loss: 0.2460 - acc: 0.9082 - val_loss: 0.2253 - val_acc: 0.9161
Epoch 19/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2438 - acc: 0.9085 - val_loss: 0.2260 - val_acc: 0.9141
Epoch 20/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2405 - acc: 0.9099 - val_loss: 0.2229 - val_acc: 0.9169
Epoch 21/50
34816/48000 [====================>.........] - ETA: 1s - loss: 0.2307 - acc: 0.9145

48000/48000 [==============================] - 4s 86us/step - loss: 0.2317 - acc: 0.9131 - val_loss: 0.2263 - val_acc: 0.9173
Epoch 22/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.2292 - acc: 0.9144 - val_loss: 0.2189 - val_acc: 0.9191
Epoch 23/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2298 - acc: 0.9141 - val_loss: 0.2282 - val_acc: 0.9139
Epoch 24/50
48000/48000 [==============================] - 4s 86us/step - loss: 0.2280 - acc: 0.9140 - val_loss: 0.2217 - val_acc: 0.9177
Epoch 25/50
34560/48000 [====================>.........] - ETA: 1s - loss: 0.2283 - acc: 0.9141

48000/48000 [==============================] - 4s 86us/step - loss: 0.2265 - acc: 0.9145 - val_loss: 0.2242 - val_acc: 0.9191
Epoch 26/50
48000/48000 [==============================] - 4s 85us/step - loss: 0.2180 - acc: 0.9191 - val_loss: 0.2354 - val_acc: 0.9123
Epoch 00026: early stopping
time elapsed: 121.86227059364319


In [7]:
print('TEST LOSS:', evaluation[0])
print('TEST ACCURACY:', evaluation[1])

TEST LOSS: 0.2467023584008217
TEST ACCURACY: 0.9116


### Bayesian Optimization

In [0]:
# Specify bounds for hyperparameters

# bounds dict should be in order of continuous type and then discrete type

bounds = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.0, 0.5)},
          {'name': 'l2_drop',          'type': 'continuous',  'domain': (0.0, 0.5)},
          {'name': 'l3_drop',          'type': 'continuous',  'domain': (0.0, 0.5)},
          {'name': 'l4_drop',          'type': 'continuous',  'domain': (0.0, 0.5)},
          {'name': 'l1_out',           'type': 'discrete',    'domain': (16, 32, 64, 128, 256, 512)},
          {'name': 'l2_out',           'type': 'discrete',    'domain': (32, 64, 128, 256, 512, 1024)},
          {'name': 'l3_out',           'type': 'discrete',    'domain': (32, 64, 128, 256, 512, 1024)},
          {'name': 'l4_out',           'type': 'discrete',    'domain': (32, 64, 128, 256, 512, 1024)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (64, 128, 256, 512)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (30, 40, 50, 60)}]

In [0]:
# function to optimize fashion mnist model

def f(x):
  
    print(x)
    
    evaluation = run_fashion_mnist(
        img_rows=28, 
        img_cols=28,
        num_classes=10,
        l1_out = int(x[:,5]),
        l2_out = int(x[:,6]),
        l3_out = int(x[:,7]),
        l4_out = int(x[:,8]),
        l1_drop = float(x[:,1]),
        l2_drop = float(x[:,2]),
        l3_drop = float(x[:,3]),
        l4_drop = float(x[:,4]),
        batch_size = int(x[:,9]),
        epochs = int(x[:,10]),
        validation_split = float(x[:,0]))
    
    print("TEST LOSS: {} \t TEST ACCURACY: {}".format(evaluation[0], evaluation[1]))
    print(evaluation, "\n")
        
    return evaluation[0]

### Optimizer instance

Install and import GPyOpt

http://sheffieldml.github.io/GPyOpt/

In [10]:
! pip install GPy
! pip install gpyopt

    100% |████████████████████████████████| 860kB 6.5MB/s 
    100% |████████████████████████████████| 71kB 9.7MB/s 
  Running setup.py bdist_wheel for GPy ... - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/36/b9/13/498e283b6081c81aded82d1c174f8c251096027be686e79005
  Running setup.py bdist_wheel for paramz ... - \ done
  Stored in directory: /content/.cache/pip/wheels/17/72/fe/035ce71322865ed7de8a98cac33e65f464dcc2bcbcf93cf642
Successfully built GPy paramz
    100% |████████████████████████████████| 61kB 2.2MB/s 
  Running setup.py bdist_wheel for gpyopt ... - \ done
  Stored in directory: /content/.cache/pip/wheels/33/1d/87/dc02440831ba986b1547dd11a7dcd44e893b0527083066d869
Successfully built gpyopt


In [0]:
import GPy, GPyOpt

In [12]:
# optimizer


# Initialize the Bayesian Optimization method.

# f: function to optimize
### should take 2-dimensional numpy arrays as input and return 2-dimensional outputs 
# domain: 
### list of dictionaries containing the description of the inputs variables 

import time
start = time.time()

optimize_fashion_mnist = GPyOpt.methods.BayesianOptimization(f=f, 
                                                             domain=bounds,
                                                             acquisition_type='EI')

end = time.time()
print("time elapsed: {}".format((end - start)/60))

# time elapsed: 27.23

[[2.98574033e-01 2.95115921e-01 2.65278698e-01 1.40351954e-01
  4.77346610e-01 2.56000000e+02 6.40000000e+01 6.40000000e+01
  1.28000000e+02 1.28000000e+02 5.00000000e+01]]
Train on 42085 samples, validate on 17915 samples
Epoch 1/50
42085/42085 [==============================] - 14s 324us/step - loss: 0.8300 - acc: 0.6960 - val_loss: 0.4606 - val_acc: 0.8262
Epoch 2/50
25984/42085 [=================>............] - ETA: 4s - loss: 0.5221 - acc: 0.8069

42085/42085 [==============================] - 13s 305us/step - loss: 0.5041 - acc: 0.8159 - val_loss: 0.3860 - val_acc: 0.8555
Epoch 3/50
42085/42085 [==============================] - 13s 305us/step - loss: 0.4269 - acc: 0.8453 - val_loss: 0.3332 - val_acc: 0.8751
Epoch 4/50
40320/42085 [===========================>..] - ETA: 0s - loss: 0.3868 - acc: 0.8590

42085/42085 [==============================] - 13s 305us/step - loss: 0.3864 - acc: 0.8595 - val_loss: 0.3251 - val_acc: 0.8776
Epoch 5/50
42085/42085 [==============================] - 13s 304us/step - loss: 0.3551 - acc: 0.8717 - val_loss: 0.2908 - val_acc: 0.8957
Epoch 6/50
42085/42085 [==============================] - 13s 305us/step - loss: 0.3342 - acc: 0.8793 - val_loss: 0.2828 - val_acc: 0.8960
Epoch 7/50
 1152/42085 [..............................] - ETA: 11s - loss: 0.3006 - acc: 0.8863

42085/42085 [==============================] - 13s 304us/step - loss: 0.3207 - acc: 0.8827 - val_loss: 0.2719 - val_acc: 0.8977
Epoch 8/50
42085/42085 [==============================] - 13s 305us/step - loss: 0.3054 - acc: 0.8879 - val_loss: 0.2691 - val_acc: 0.8996
Epoch 9/50
34688/42085 [=======================>......] - ETA: 2s - loss: 0.2967 - acc: 0.8903

42085/42085 [==============================] - 13s 306us/step - loss: 0.2972 - acc: 0.8908 - val_loss: 0.2684 - val_acc: 0.9007
Epoch 10/50
42085/42085 [==============================] - 13s 304us/step - loss: 0.2899 - acc: 0.8942 - val_loss: 0.2575 - val_acc: 0.9034
Epoch 11/50
42085/42085 [==============================] - 13s 305us/step - loss: 0.2801 - acc: 0.8980 - val_loss: 0.2562 - val_acc: 0.9069
Epoch 12/50
  128/42085 [..............................] - ETA: 13s - loss: 0.2084 - acc: 0.8984

...

45396/45396 [==============================] - 17s 367us/step - loss: 0.2250 - acc: 0.9173 - val_loss: 0.2523 - val_acc: 0.9078
Epoch 14/40
45396/45396 [==============================] - 17s 367us/step - loss: 0.2167 - acc: 0.9192 - val_loss: 0.2493 - val_acc: 0.9081
Epoch 15/40
17344/45396 [==========>...................] - ETA: 9s - loss: 0.2095 - acc: 0.9216

45396/45396 [==============================] - 17s 367us/step - loss: 0.2137 - acc: 0.9204 - val_loss: 0.2627 - val_acc: 0.9061
Epoch 16/40
45396/45396 [==============================] - 17s 368us/step - loss: 0.2069 - acc: 0.9216 - val_loss: 0.2440 - val_acc: 0.9098
Epoch 17/40
16960/45396 [==========>...................] - ETA: 9s - loss: 0.2009 - acc: 0.9270

45396/45396 [==============================] - 17s 366us/step - loss: 0.2031 - acc: 0.9249 - val_loss: 0.2401 - val_acc: 0.9132
Epoch 18/40
45396/45396 [==============================] - 17s 366us/step - loss: 0.1959 - acc: 0.9266 - val_loss: 0.2479 - val_acc: 0.9116
Epoch 19/40
17600/45396 [==========>...................] - ETA: 9s - loss: 0.1908 - acc: 0.9299

45396/45396 [==============================] - 17s 367us/step - loss: 0.1928 - acc: 0.9279 - val_loss: 0.2455 - val_acc: 0.9125
Epoch 20/40
45396/45396 [==============================] - 17s 367us/step - loss: 0.1911 - acc: 0.9287 - val_loss: 0.2594 - val_acc: 0.9076
Epoch 21/40
17408/45396 [==========>...................] - ETA: 9s - loss: 0.1815 - acc: 0.9309

45396/45396 [==============================] - 17s 368us/step - loss: 0.1853 - acc: 0.9306 - val_loss: 0.2609 - val_acc: 0.9095
Epoch 00021: early stopping
TEST LOSS: 0.27149430224895477 	 TEST ACCURACY: 0.9082
[0.27149430224895477, 0.9082] 

time elapsed: 19.132859774430592


### Running Optmization

In [0]:
# optimize mnist model

start = time.time()

optimize_fashion_mnist.run_optimization(max_iter=10)

end = time.time()
print("time elapsed: {}".format((end - start)/60))

[[6.45471430e-02 5.22864772e-03 7.14942354e-03 5.00000000e-01
  3.23694978e-01 6.40000000e+01 3.20000000e+01 6.40000000e+01
  1.28000000e+02 5.12000000e+02 3.00000000e+01]]
Train on 56127 samples, validate on 3873 samples
Epoch 1/30
56127/56127 [==============================] - 5s 95us/step - loss: 0.9147 - acc: 0.6635 - val_loss: 0.4944 - val_acc: 0.8120
Epoch 2/30
56127/56127 [==============================] - 4s 78us/step - loss: 0.5297 - acc: 0.8041 - val_loss: 0.3909 - val_acc: 0.8613
Epoch 3/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.4503 - acc: 0.8378 - val_loss: 0.3504 - val_acc: 0.8683
Epoch 4/30
30208/56127 [===============>..............] - ETA: 1s - loss: 0.4137 - acc: 0.8512

56127/56127 [==============================] - 4s 77us/step - loss: 0.4078 - acc: 0.8535 - val_loss: 0.3104 - val_acc: 0.8823
Epoch 5/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.3768 - acc: 0.8636 - val_loss: 0.3039 - val_acc: 0.8872
Epoch 6/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.3570 - acc: 0.8698 - val_loss: 0.2877 - val_acc: 0.8928
Epoch 7/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.3385 - acc: 0.8765 - val_loss: 0.2766 - val_acc: 0.8993
Epoch 8/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.3227 - acc: 0.8824 - val_loss: 0.2800 - val_acc: 0.8975


Epoch 9/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.3131 - acc: 0.8858 - val_loss: 0.2640 - val_acc: 0.8985
Epoch 10/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.3036 - acc: 0.8903 - val_loss: 0.2669 - val_acc: 0.9006
Epoch 11/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.2946 - acc: 0.8933 - val_loss: 0.2537 - val_acc: 0.9096
Epoch 12/30
56127/56127 [==============================] - 4s 78us/step - loss: 0.2919 - acc: 0.8922 - val_loss: 0.2535 - val_acc: 0.9050
Epoch 13/30
48640/56127 [========================>.....] - ETA: 0s - loss: 0.2825 - acc: 0.8965

56127/56127 [==============================] - 4s 75us/step - loss: 0.2821 - acc: 0.8968 - val_loss: 0.2484 - val_acc: 0.9070
Epoch 14/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.2756 - acc: 0.8986 - val_loss: 0.2578 - val_acc: 0.9014
Epoch 15/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.2671 - acc: 0.9024 - val_loss: 0.2423 - val_acc: 0.9104
Epoch 16/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.2628 - acc: 0.9036 - val_loss: 0.2349 - val_acc: 0.9125
Epoch 17/30
56127/56127 [==============================] - 4s 80us/step - loss: 0.2549 - acc: 0.9075 - val_loss: 0.2281 - val_acc: 0.9125
Epoch 18/30
  512/56127 [..............................] - ETA: 4s - loss: 0.3201 - acc: 0.8848

56127/56127 [==============================] - 4s 77us/step - loss: 0.2530 - acc: 0.9078 - val_loss: 0.2263 - val_acc: 0.9171
Epoch 19/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.2503 - acc: 0.9086 - val_loss: 0.2271 - val_acc: 0.9161
Epoch 20/30
56127/56127 [==============================] - 4s 76us/step - loss: 0.2431 - acc: 0.9097 - val_loss: 0.2257 - val_acc: 0.9138
Epoch 21/30
56127/56127 [==============================] - 4s 77us/step - loss: 0.2380 - acc: 0.9126 - val_loss: 0.2200 - val_acc: 0.9192
Epoch 22/30
52736/56127 [===========================>..] - ETA: 0s - loss: 0.2373 - acc: 0.9135

...

55097/55097 [==============================] - 3s 60us/step - loss: 0.5841 - acc: 0.7807 - val_loss: 0.4424 - val_acc: 0.8409
Epoch 3/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.5104 - acc: 0.8131 - val_loss: 0.3947 - val_acc: 0.8572
Epoch 4/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.4645 - acc: 0.8306 - val_loss: 0.3629 - val_acc: 0.8642
Epoch 5/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.4339 - acc: 0.8425 - val_loss: 0.3419 - val_acc: 0.8744
Epoch 6/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.4048 - acc: 0.8520 - val_loss: 0.3343 - val_acc: 0.8801
Epoch 7/50
 5632/55097 [==>...........................] - ETA: 2s - loss: 0.3898 - acc: 0.8560

55097/55097 [==============================] - 3s 60us/step - loss: 0.3865 - acc: 0.8593 - val_loss: 0.3133 - val_acc: 0.8827
Epoch 8/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3681 - acc: 0.8651 - val_loss: 0.3049 - val_acc: 0.8874
Epoch 9/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3524 - acc: 0.8707 - val_loss: 0.2877 - val_acc: 0.8913
Epoch 10/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3433 - acc: 0.8737 - val_loss: 0.2807 - val_acc: 0.8956
Epoch 11/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3321 - acc: 0.8788 - val_loss: 0.2831 - val_acc: 0.8944
Epoch 12/50
 1536/55097 [..............................] - ETA: 3s - loss: 0.3431 - acc: 0.8737

55097/55097 [==============================] - 3s 60us/step - loss: 0.3233 - acc: 0.8824 - val_loss: 0.2678 - val_acc: 0.9001
Epoch 13/50
55097/55097 [==============================] - 3s 59us/step - loss: 0.3182 - acc: 0.8838 - val_loss: 0.2700 - val_acc: 0.8978
Epoch 14/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3103 - acc: 0.8885 - val_loss: 0.2673 - val_acc: 0.8990
Epoch 15/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.3014 - acc: 0.8906 - val_loss: 0.2557 - val_acc: 0.9058
Epoch 16/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2946 - acc: 0.8926 - val_loss: 0.2560 - val_acc: 0.9033


Epoch 17/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2898 - acc: 0.8945 - val_loss: 0.2450 - val_acc: 0.9082
Epoch 18/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2860 - acc: 0.8954 - val_loss: 0.2452 - val_acc: 0.9101
Epoch 19/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2774 - acc: 0.8984 - val_loss: 0.2423 - val_acc: 0.9092
Epoch 20/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2764 - acc: 0.8990 - val_loss: 0.2493 - val_acc: 0.9068
Epoch 21/50
51712/55097 [===========================>..] - ETA: 0s - loss: 0.2693 - acc: 0.9005

55097/55097 [==============================] - 3s 60us/step - loss: 0.2697 - acc: 0.9004 - val_loss: 0.2373 - val_acc: 0.9111
Epoch 22/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2687 - acc: 0.9015 - val_loss: 0.2349 - val_acc: 0.9088
Epoch 23/50
55097/55097 [==============================] - 3s 60us/step - loss: 0.2630 - acc: 0.9025 - val_loss: 0.2335 - val_acc: 0.9147
Epoch 24/50
45568/55097 [=======================>......] - ETA: 0s - loss: 0.2564 - acc: 0.9051

In [64]:
# Show optimized fashion mnist model


print("""
Optimized Parameters:\n 
\t{0}: {1}
\t{2}: {3}
\t{4}: {5}
\t{6}: {7}
\t{8}: {9}
\t{10}: {11}
\t{12}: {13}
\t{14}: {15}
\t{16}: {17}
\t{18}: {19}
\t{20}: {21}
""".format(bounds[0]["name"], optimize_fashion_mnist.x_opt[0],
           bounds[1]["name"], optimize_fashion_mnist.x_opt[1],
           bounds[2]["name"], optimize_fashion_mnist.x_opt[2],
           bounds[3]["name"], optimize_fashion_mnist.x_opt[3],
           bounds[4]["name"], optimize_fashion_mnist.x_opt[4],
           bounds[5]["name"], optimize_fashion_mnist.x_opt[5],
           bounds[6]["name"], optimize_fashion_mnist.x_opt[6],
           bounds[7]["name"], optimize_fashion_mnist.x_opt[7],
           bounds[8]["name"], optimize_fashion_mnist.x_opt[8],
           bounds[9]["name"], optimize_fashion_mnist.x_opt[9],
           bounds[10]["name"], optimize_fashion_mnist.x_opt[10]))

print("optimized loss: {0}".format(optimize_fashion_mnist.fx_opt))


Optimized Parameters: 
	validation_split: 0.09159103468674865
	l1_drop: 0.20775426571420313
	l2_drop: 0.29613491912015927
	l3_drop: 0.06437685604642024
	l4_drop: 0.3545708336915873
	l1_out: 16.0
	l2_out: 128.0
	l3_out: 128.0
	l4_out: 64.0
	batch_size: 256.0
	epochs: 60.0

optimized loss: 0.22566337995529176


In [7]:
fashion_mnist_opt = FASHION_MNIST(img_rows=28, 
                              img_cols=28,
                              num_classes=10,
                              l1_out = int(optimize_fashion_mnist.x_opt[5]),
                              l2_out = int(optimize_fashion_mnist.x_opt[6]),
                              l3_out = int(optimize_fashion_mnist.x_opt[7]),
                              l4_out = int(optimize_fashion_mnist.x_opt[8]),
                              l1_drop = float(optimize_fashion_mnist.x_opt[1]),
                              l2_drop = float(optimize_fashion_mnist.x_opt[2]),
                              l3_drop = float(optimize_fashion_mnist.x_opt[3]),
                              l4_drop = float(optimize_fashion_mnist.x_opt[4]),
                              batch_size = int(optimize_fashion_mnist.x_opt[9]),
                              epochs = int(optimize_fashion_mnist.x_opt[10]),
                              validation_split = float(optimize_fashion_mnist.x_opt[0]))


history = fashion_mnist_opt.fit()

evaluation = fashion_mnist_opt.evaluate()

NameError: ignored

In [68]:
evaluation = run_fashion_mnist(img_rows=28, 
                              img_cols=28,
                              num_classes=10,
                              l1_out = int(optimize_fashion_mnist.x_opt[5]),
                              l2_out = int(optimize_fashion_mnist.x_opt[6]),
                              l3_out = int(optimize_fashion_mnist.x_opt[7]),
                              l4_out = int(optimize_fashion_mnist.x_opt[8]),
                              l1_drop = float(optimize_fashion_mnist.x_opt[1]),
                              l2_drop = float(optimize_fashion_mnist.x_opt[2]),
                              l3_drop = float(optimize_fashion_mnist.x_opt[3]),
                              l4_drop = float(optimize_fashion_mnist.x_opt[4]),
                              batch_size = int(optimize_fashion_mnist.x_opt[9]),
                              epochs = int(optimize_fashion_mnist.x_opt[10]),
                              validation_split = float(optimize_fashion_mnist.x_opt[0]))


Train on 54504 samples, validate on 5496 samples
Epoch 1/60
54504/54504 [==============================] - 9s 171us/step - loss: 0.8382 - acc: 0.6924 - val_loss: 0.5015 - val_acc: 0.8153
Epoch 2/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.5250 - acc: 0.8077 - val_loss: 0.4088 - val_acc: 0.8437
Epoch 3/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.4591 - acc: 0.8340 - val_loss: 0.3592 - val_acc: 0.8701
Epoch 4/60
43264/54504 [======================>.......] - ETA: 0s - loss: 0.4139 - acc: 0.8502

54504/54504 [==============================] - 5s 88us/step - loss: 0.4123 - acc: 0.8504 - val_loss: 0.3341 - val_acc: 0.8774
Epoch 5/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.3818 - acc: 0.8629 - val_loss: 0.3123 - val_acc: 0.8866
Epoch 6/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.3569 - acc: 0.8711 - val_loss: 0.2908 - val_acc: 0.8934
Epoch 7/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.3407 - acc: 0.8802 - val_loss: 0.2766 - val_acc: 0.8990
Epoch 8/60
20992/54504 [==========>...................] - ETA: 2s - loss: 0.3212 - acc: 0.8861

54504/54504 [==============================] - 5s 87us/step - loss: 0.3222 - acc: 0.8849 - val_loss: 0.2684 - val_acc: 0.9030
Epoch 9/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.3064 - acc: 0.8889 - val_loss: 0.2611 - val_acc: 0.9019
Epoch 10/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.2994 - acc: 0.8925 - val_loss: 0.2607 - val_acc: 0.9034
Epoch 11/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2867 - acc: 0.8964 - val_loss: 0.2495 - val_acc: 0.9052
Epoch 12/60
17664/54504 [========>.....................] - ETA: 3s - loss: 0.2737 - acc: 0.9023

54504/54504 [==============================] - 5s 87us/step - loss: 0.2781 - acc: 0.9002 - val_loss: 0.2441 - val_acc: 0.9110
Epoch 13/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2675 - acc: 0.9033 - val_loss: 0.2421 - val_acc: 0.9108
Epoch 14/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.2625 - acc: 0.9046 - val_loss: 0.2348 - val_acc: 0.9116
Epoch 15/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2567 - acc: 0.9066 - val_loss: 0.2441 - val_acc: 0.9107
Epoch 16/60
14080/54504 [======>.......................] - ETA: 3s - loss: 0.2523 - acc: 0.9073

54504/54504 [==============================] - 5s 88us/step - loss: 0.2496 - acc: 0.9090 - val_loss: 0.2376 - val_acc: 0.9107
Epoch 17/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2422 - acc: 0.9099 - val_loss: 0.2371 - val_acc: 0.9138
Epoch 18/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.2379 - acc: 0.9138 - val_loss: 0.2307 - val_acc: 0.9143
Epoch 19/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2353 - acc: 0.9131 - val_loss: 0.2232 - val_acc: 0.9179
Epoch 20/60
13312/54504 [======>.......................] - ETA: 3s - loss: 0.2256 - acc: 0.9188

54504/54504 [==============================] - 5s 88us/step - loss: 0.2255 - acc: 0.9175 - val_loss: 0.2248 - val_acc: 0.9118
Epoch 21/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2230 - acc: 0.9182 - val_loss: 0.2225 - val_acc: 0.9176
Epoch 22/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2185 - acc: 0.9196 - val_loss: 0.2131 - val_acc: 0.9179
Epoch 23/60
54504/54504 [==============================] - 5s 88us/step - loss: 0.2137 - acc: 0.9215 - val_loss: 0.2172 - val_acc: 0.9198
Epoch 24/60
14080/54504 [======>.......................] - ETA: 3s - loss: 0.2039 - acc: 0.9268

54504/54504 [==============================] - 5s 88us/step - loss: 0.2081 - acc: 0.9241 - val_loss: 0.2188 - val_acc: 0.9179
Epoch 25/60
54504/54504 [==============================] - 5s 87us/step - loss: 0.2088 - acc: 0.9228 - val_loss: 0.2214 - val_acc: 0.9187
Epoch 26/60
54504/54504 [==============================] - 5s 89us/step - loss: 0.2056 - acc: 0.9241 - val_loss: 0.2185 - val_acc: 0.9181
Epoch 00026: early stopping


In [69]:
evaluation

[0.23914691522121428, 0.9144]

In [0]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)